In [12]:
import gensim
import sklearn
from sklearn.externals import joblib
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import svm
import pandas as pd
import numpy as np

In [2]:
model_russian = gensim.models.KeyedVectors.load_word2vec_format('vectors/wiki.ru.vec')

In [ ]:
del model

In [3]:

model = gensim.models.KeyedVectors.load_word2vec_format('vectors/GoogleNews-vectors-negative300.bin', binary=True)

In [36]:
def vectorize_word(word):
	if word in model.wv:
		return pd.Series(model.wv[word])
	else:
		# The pretrained word2vec model has dimensionality 300
		return pd.Series([0] * 300)
def merge_vectors(v1, v2, method):
	print "method", method
	# Concat
	if method == 'concat':
		return pd.concat([v1, v2, y], axis = 1)
	elif method == 'diff':
		# Roller 2014 says to normalize the difference
		diff = v1 - v2
		return pd.concat([diff, y], axis = 1)
	elif method == 'asym':
		print "asym"
		# diff
		a = v1 - v2
		# squared diff - can't tell if they mean the mag^2 or ea element sq?
		b = pd.DataFrame(np.sqrt(np.square(a.values).sum(axis=1)))
		return pd.concat([a, b, y], axis = 1)
	elif method == 'cosine':
		#this fails???
		print "cosine"
		cos = pd.DataFrame(cosine_similarity(vectors_0.values, vectors_1.values).diagonal())
		return pd.concat([cos, y], axis = 1)

In [206]:
from sklearn.externals import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

clf = joblib.load('models/svm.pkl') 

folder = 'baroni2012'
test_vectorized = pd.read_csv('lexical_entailment/'+folder+'/data_lex_test_vectorized.tsv', sep='\t', header=None)
val_vectorized = pd.read_csv('lexical_entailment/'+folder+'/data_lex_val_vectorized.tsv', sep='\t', header=None)

for test_name, test_df in zip(['test', 'val'], [test_vectorized, val_vectorized]):
	orig_rows, orig_cols = test_df.shape

	# Remove rows with NaN
	test_df.dropna(axis=0, inplace=True)

	# Count number of rows removed
	diff = orig_rows - test_df.shape[0]

	X = test_df.iloc[:, :-1]
	y = test_df.iloc[:, -1]

	preds = clf.predict(X)

	true_count = y.sum()
	preds_count = preds.sum()

	print "precision", metrics.precision_score(y, preds)
	print "recall", metrics.recall_score(y, preds)
	print "f1", metrics.f1_score(y, preds)
	print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
	print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])
	print true_count, preds_count, orig_rows

	num_correct = metrics.accuracy_score(y, preds, normalize=False)

	print test_name, ": percentage non-nan correct:", num_correct/float(test_df.shape[0]) 
	print test_name, ": percentage correct overall", num_correct/float(orig_rows)
    

precision 0.71186440678
recall 0.158490566038
f1 0.259259259259
True 0.158490566038
False 0.93536121673
265 59 536
test : percentage non-nan correct: 0.545454545455
test : percentage correct overall 0.537313432836
precision 0.947368421053
recall 0.409090909091
f1 0.571428571429
True 0.409090909091
False 0.976744186047
44 19 87
val : percentage non-nan correct: 0.689655172414
val : percentage correct overall 0.689655172414


In [37]:
folder = 'bless2011'
method = 'cosine'
for subset in ['test', 'train', 'val']:
    path_to_dataset = 'lexical_entailment/' + folder + '/data_lex_' + subset + '.tsv'
    path_to_vectorized = 'lexical_entailment/' + folder + '/data_lex_' + subset + '_vectorized_cosine.tsv'
    df = pd.read_csv(path_to_dataset, sep='\t', header=None)
    words_0 = df[0]
    words_1 = df[1]
    y = df[[2]]
    vectors_0 = words_0.apply(vectorize_word)
    vectors_1 = words_1.apply(vectorize_word)
    vectors_x = merge_vectors(vectors_0[:300], vectors_1[:300], method)
    vectors_x.to_csv(path_to_vectorized, sep='\t', header=False, index=False)



method cosine
cosine
method cosine
cosine
method cosine
cosine


In [72]:
#Train cosine 
df = pd.read_csv("lexical_entailment/bless2011/data_lex_train_vectorized_cosine.tsv", sep='\t', header=None)
df = df[df[0] != 0.0]
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
clf = svm.LinearSVC(class_weight='balanced')
clf.fit(X,y)
joblib.dump(clf, 'models/svm_cosine.pkl')

['models/svm_cosine.pkl']

['models/svm_cosine.pkl']

In [208]:
test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized_asym.tsv', sep='\t', header=None)
test = pd.read_csv('lexical_entailment/bless2011/data_lex_test.tsv', sep='\t', header=None)
val_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_val_vectorized_asym.tsv', sep='\t', header=None)

In [216]:
train_df = val_vectorized
test_df = test_vectorized
train_df.dropna(axis=0, inplace=True)
test_df.dropna(axis=0, inplace=True)
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1]

In [222]:
val_vectorized

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,300,301
0,-0.189951,0.792969,0.274010,-0.191406,-0.094727,-0.271973,-0.330566,0.040039,0.878906,0.315613,...,0.244141,-0.003906,0.444336,0.360352,-0.206909,-0.294678,-0.219238,-0.107422,5.551709,False
1,-0.109375,0.385742,-0.437500,-0.106445,0.232422,-0.273926,-0.333984,0.144531,0.205688,-0.072754,...,0.386719,0.303894,0.418945,0.106689,0.008789,0.203613,0.397949,-0.028381,5.354463,False
2,-0.494141,0.117798,0.272949,0.127930,0.374023,0.183105,-0.137695,-0.200439,-0.036346,0.378906,...,0.309082,-0.293884,0.640137,-0.087891,-0.448242,0.345703,-0.207520,-0.123779,4.695697,False
3,-0.142578,0.088379,-0.107300,0.243652,0.500977,0.161377,-0.187500,-0.351074,0.105469,0.010742,...,-0.075684,0.258202,0.279297,-0.016113,0.146118,0.406494,0.583984,0.094727,4.308543,False
4,0.001953,0.019348,-0.297607,-0.033203,-0.736328,0.467773,0.066406,0.133667,-0.243652,0.168945,...,-0.299805,0.354004,0.183105,-0.282715,-0.025391,0.602051,-0.449219,-0.274170,4.718472,False
5,-0.301025,0.156250,-0.045746,-0.521484,0.223022,-0.323730,-0.316406,-0.219727,0.183838,0.059570,...,0.074219,0.404785,0.371094,-0.646484,-0.136719,-0.054626,0.233459,0.282776,4.398509,False
6,0.214920,-0.186890,-0.052734,0.552002,0.012451,0.021240,0.493164,0.130371,-0.139404,-0.028320,...,0.119957,0.010254,0.104004,0.077148,-0.446777,0.166504,0.031250,-0.117554,3.311437,False
7,-0.016602,0.032349,-0.043152,0.013672,-0.045166,0.236816,0.118652,-0.065430,0.076599,0.111328,...,0.254883,0.121094,0.025391,-0.146973,0.238403,0.000183,-0.059082,-0.202148,2.904589,False
9,0.235352,-0.097046,0.477539,0.016602,-0.464844,0.182373,0.097168,-0.309570,-0.084473,0.038330,...,-0.040039,-0.452148,0.120850,0.007812,-0.083984,-0.144165,0.337891,-0.100586,4.724591,False
10,0.071657,0.064453,-0.406250,0.151855,-0.006836,-0.112305,-0.120117,0.408600,0.007812,-0.048828,...,-0.274902,-0.329102,0.197266,-0.074829,-0.276367,0.127197,0.083862,0.133789,3.674977,False


In [217]:

preds = clf.predict(X)

In [211]:
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])

precision 0.806451612903
recall 0.892857142857
f1 0.847457627119
True 0.892857142857
False 0.980456026059


In [202]:
p_hat = clf.coef_[0][301:]

In [212]:
def normalize(v):
  '''normalize' a vector, in the traditional linear algebra sense.'''
  norm=np.linalg.norm(v)
  if norm==0:
    return v
  return v/norm
def reject(A):

  '''Create a 'projection', and subract it from the original vector'''
  B = p_hat
  project = np.linalg.linalg.dot(A, normalize(B)) * normalize(B)
  return A - project


In [213]:
X_hat = X.apply(reject, axis=1)

ValueError: ('shapes (301,) and (0,) not aligned: 301 (dim 0) != 0 (dim 0)', u'occurred at index 0')

In [108]:
clf_hat = svm.LinearSVC()
clf_hat.fit(X_hat, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [109]:
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]
preds_hat = clf_hat.predict(X_test)
print "precision", metrics.precision_score(y_test, preds_hat)
print "recall", metrics.recall_score(y_test, preds_hat)
print "f1", metrics.f1_score(y_test, preds_hat)
print "True", metrics.accuracy_score(y_test[y_test == 1], preds_hat[y_test == 1])
print "False", metrics.accuracy_score(y_test[y_test == 0], preds_hat[y_test == 0])

precision 0.416243654822
recall 0.30258302583
f1 0.350427350427
True 0.30258302583
False 0.964788732394


In [110]:
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])

precision 0.806451612903
recall 0.892857142857
f1 0.847457627119
True 0.892857142857
False 0.980456026059


In [157]:
clf = joblib.load('models/svm.pkl') 

In [170]:
fp = 'lexical_entailment/bless2011/data_lex_test.tsv'
test = pd.read_csv(fp, sep='\t', header=None)
test_vectorized = pd.read_csv('lexical_entailment/bless2011/data_lex_test_vectorized.tsv', sep='\t', header=None)

In [171]:

test_df = test_vectorized
test_df.dropna(axis=0, inplace=True)

X = test_df.iloc[:, :-1]
y = test_df.iloc[:, -1]

preds = clf.predict(X)
true_count = y.sum()
preds_count = preds.sum()
print "precision", metrics.precision_score(y, preds)
print "recall", metrics.recall_score(y, preds)
print "f1", metrics.f1_score(y, preds)
print "True", metrics.accuracy_score(y[y == 1], preds[y == 1])
print "False", metrics.accuracy_score(y[y == 0], preds[y == 0])
print true_count, preds_count, orig_rows

num_correct = metrics.accuracy_score(y, preds, normalize=False)

print test_name, ": percentage non-nan correct:", num_correct/float(test_df.shape[0])
print test_name, ": percentage correct overall", num_correct/float(orig_rows)

precision 0.406113537118
recall 0.343173431734
f1 0.372
True 0.343173431734
False 0.958358848745
271 229 358
val : percentage non-nan correct: 0.911224201301
val : percentage correct overall 9.00279329609


In [35]:
#Vectorize train/test/val sets

path_to_dataset = 'lexical_entailment/turney2014/data_lex_val.tsv'
path_to_vectorized = 'lexical_entailment/turney2014/data_lext_val_vectorized.tsv'
df = pd.read_csv(path_to_dataset, sep='\t', header=None)

words_0 = df[0]
words_1 = df[1]
y = df[[2]]
method = 'cosine'
def vectorize_word(word):
	if word in model.wv:
		return pd.Series(model.wv[word])
	else:
		# The pretrained word2vec model has dimensionality 300
		return pd.Series([np.nan] * 300)

vectors_0 = words_0.apply(vectorize_word)
vectors_1 = words_1.apply(vectorize_word)
def merge_vectors(v1, v2, method):
	print "method", method
	# Concat
	if method == 'concat':
		return pd.concat([vectors_0, vectors_1, y], axis = 1)
	elif method == 'diff':
		# Roller 2014 says to normalize the difference
		diff = vectors_0 - vectors_1
		return pd.concat([diff, y], axis = 1)
	elif method == 'asym':
		print "asym"
		# diff
		a = vectors_0 - vectors_1
		# squared diff - can't tell if they mean the mag^2 or ea element sq?
		b = pd.DataFrame(np.sqrt(np.square(a.values).sum(axis=1)))
		return pd.concat([a, b, y], axis = 1)
	elif method == 'cosine':
		#this fails???
		print "cosine"
		cos = pd.DataFrame(cosine_similarity(vectors_0.values, vectors_1.values).diagonal())
		return pd.concat([v1, v2, cos, y], axis = 1)

vectors_x = merge_vectors(vectors_0, vectors_1, method)
vectors_x.to_csv(path_to_vectorized, sep='\t', header=False, index=False)

method cosine
cosine


In [151]:
X

,0,1
0,absentia,sentence
1,accounting,management
2,activist,federation
3,advisories,federation
4,aerospace,management
5,agency,government
6,agriculture,environment
7,agriculture,management
8,ambassador,chairman
9,amendment,bill


In [33]:
fp = "lexical_entailment/bless2011/data_lex_val_vectorized_cosine.tsv"
df = pd.read_csv(fp,sep='\t')
for index, row in df.iterrows():
    print row


0.000478744506836      0.115234
0.51953125             0.326172
-0.00138092041016     -0.245117
0.126953125           -0.205078
-0.142578125           0.408203
-0.28125              -0.178711
-0.03369140625        -0.177734
-0.2041015625         -0.234375
0.59765625             0.183594
0.310546875            0.106934
-0.00381469726562      0.150391
-0.373046875          -0.523438
0.007568359375      -0.00622559
0.177734375           -0.183594
0.053466796875       -0.0961914
0.412109375            0.648438
-0.1669921875         -0.144531
0.177734375.1         -0.123535
0.06640625            0.0559082
-0.5625               -0.231445
-0.126953125           0.384766
-0.150390625          0.0253906
-0.1259765625          0.168945
-0.248046875         -0.0961914
0.3046875              0.330078
0.0140991210938            -0.5
-0.0859375             0.126953
-0.0220947265625       0.482422
0.076171875          -0.0284424
-0.3984375           -0.0289307
                        ...    
0.190429

0.000478744506836     0.318359
0.51953125            0.279297
-0.00138092041016    -0.166016
0.126953125           0.214844
-0.142578125          0.216797
-0.28125              0.185547
-0.03369140625       -0.203125
-0.2041015625       -0.0673828
0.59765625            0.177734
0.310546875           0.105957
-0.00381469726562    -0.081543
-0.373046875         -0.269531
0.007568359375        0.152344
0.177734375           0.210938
0.053466796875       -0.617188
0.412109375           0.273438
-0.1669921875        0.0170898
0.177734375.1        -0.109375
0.06640625             0.15625
-0.5625              -0.174805
-0.126953125         0.0437012
-0.150390625         -0.133789
-0.1259765625        0.0476074
-0.248046875         0.0415039
0.3046875            0.0476074
0.0140991210938      -0.404297
-0.0859375           -0.417969
-0.0220947265625     -0.128906
0.076171875           0.320312
-0.3984375          -0.0395508
                       ...    
0.1904296875.1       0.0825195
0.157226

0.000478744506836   -0.0559082
0.51953125            0.330078
-0.00138092041016    -0.396484
0.126953125          0.0174561
-0.142578125        -0.0927734
-0.28125             -0.180664
-0.03369140625        0.308594
-0.2041015625         0.103516
0.59765625           -0.179688
0.310546875          -0.235352
-0.00381469726562     0.224609
-0.373046875          -0.28125
0.007568359375       -0.146484
0.177734375          -0.365234
0.053466796875       -0.103516
0.412109375            -0.0625
-0.1669921875        -0.180664
0.177734375.1        -0.171875
0.06640625           -0.210938
-0.5625              0.0272217
-0.126953125        -0.0249023
-0.150390625         -0.169922
-0.1259765625         0.265625
-0.248046875          0.192383
0.3046875           -0.0986328
0.0140991210938         0.0625
-0.0859375           -0.229492
-0.0220947265625      0.120117
0.076171875           0.126953
-0.3984375           -0.539062
                       ...    
0.1904296875.1       -0.231445
0.157226

Name: 166, Length: 602, dtype: object
0.000478744506836    -0.316406
0.51953125            0.123047
-0.00138092041016    0.0177002
0.126953125           0.105957
-0.142578125         0.0693359
-0.28125            -0.0378418
-0.03369140625       -0.165039
-0.2041015625         -0.28125
0.59765625            0.232422
0.310546875           0.227539
-0.00381469726562   -0.0544434
-0.373046875         -0.019165
0.007568359375      -0.0292969
0.177734375          -0.144531
0.053466796875       -0.163086
0.412109375          0.0776367
-0.1669921875        -0.171875
0.177734375.1         0.052002
0.06640625           -0.349609
-0.5625               0.306641
-0.126953125          0.106445
-0.150390625        -0.0791016
-0.1259765625         0.275391
-0.248046875          0.306641
0.3046875             0.166016
0.0140991210938      -0.294922
-0.0859375            0.140625
-0.0220947265625      0.339844
0.076171875           0.267578
-0.3984375           0.0078125
                       ...    
0

0.000478744506836     0.0123291
0.51953125             0.204102
-0.00138092041016     -0.285156
0.126953125            0.216797
-0.142578125           0.118164
-0.28125              0.0830078
-0.03369140625        0.0498047
-0.2041015625       -0.00952148
0.59765625             0.220703
0.310546875           -0.125977
-0.00381469726562     0.0805664
-0.373046875          -0.585938
0.007568359375      -0.00445557
0.177734375           -0.296875
0.053466796875       -0.0131226
0.412109375          -0.0834961
-0.1669921875         0.0505371
0.177734375.1          0.151367
0.06640625            -0.449219
-0.5625              -0.0135498
-0.126953125           0.214844
-0.150390625          -0.147461
-0.1259765625          0.224609
-0.248046875             -0.125
0.3046875             -0.097168
0.0140991210938        0.249023
-0.0859375            -0.289062
-0.0220947265625       0.365234
0.076171875            0.412109
-0.3984375           -0.0859375
                        ...    
0.190429

0.000478744506836             0
0.51953125                    0
-0.00138092041016             0
0.126953125                   0
-0.142578125                  0
-0.28125                      0
-0.03369140625                0
-0.2041015625                 0
0.59765625                    0
0.310546875                   0
-0.00381469726562             0
-0.373046875                  0
0.007568359375                0
0.177734375                   0
0.053466796875                0
0.412109375                   0
-0.1669921875                 0
0.177734375.1                 0
0.06640625                    0
-0.5625                       0
-0.126953125                  0
-0.150390625                  0
-0.1259765625                 0
-0.248046875                  0
0.3046875                     0
0.0140991210938               0
-0.0859375                    0
-0.0220947265625              0
0.076171875                   0
-0.3984375                    0
                        ...    
0.190429

0.000478744506836          NaN
0.51953125                 NaN
-0.00138092041016          NaN
0.126953125                NaN
-0.142578125               NaN
-0.28125                   NaN
-0.03369140625             NaN
-0.2041015625              NaN
0.59765625                 NaN
0.310546875                NaN
-0.00381469726562          NaN
-0.373046875               NaN
0.007568359375             NaN
0.177734375                NaN
0.053466796875             NaN
0.412109375                NaN
-0.1669921875              NaN
0.177734375.1              NaN
0.06640625                 NaN
-0.5625                    NaN
-0.126953125               NaN
-0.150390625               NaN
-0.1259765625              NaN
-0.248046875               NaN
0.3046875                  NaN
0.0140991210938            NaN
-0.0859375                 NaN
-0.0220947265625           NaN
0.076171875                NaN
-0.3984375                 NaN
                       ...    
0.1904296875.1             NaN
0.157226